# Setup

In [1]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/My\ Drive/CLIPProj

Mounted at /content/drive
/content/drive/My Drive/CLIPProj


In [2]:
# Install dependencies
# !pip freeze | grep tqdm  # check existing libraries
!pip install -r requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 818.9/818.9 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 34.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 69.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 961.5/961.5 kB 41.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
# Library Imports
import os
from dotenv import load_dotenv

import numpy as np
import matplotlib.pyplot as plt

import torch
from torch.utils.data import DataLoader, Dataset, random_split
# import torch_xla.core.xla_model as xm
# import torch_xla.distributed.xla_multiprocessing as xmp

import torchvision
from torchvision import transforms

import lightning as L
from lightning.pytorch.callbacks import ModelCheckpoint, EarlyStopping
from tqdm import tqdm

from transformers import CLIPProcessor, CLIPVisionModel

In [4]:
# General Setup
RAND_SEED = 42
load_dotenv()
L.seed_everything(RAND_SEED)

# Use Colab GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

INFO: Seed set to 42
INFO:lightning.fabric.utilities.seed:Seed set to 42


# Data

## Downloading Data

In [5]:
class ColorFix(object):
    def __call__(self, img):
        if img.mode == 'L':
            img = img.convert("RGB")
        return img

transform = transforms.Compose(
    [
        transforms.Resize((224, 224)),
        ColorFix(),
        transforms.ToTensor(),
    ]
)

In [6]:
# Caltech 101 Data
data = torchvision.datasets.Caltech101('data', transform=transform, download=True)

## CLIP Embeddings

In [7]:
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
clip_model = CLIPVisionModel.from_pretrained("openai/clip-vit-base-patch32").to(device)

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [8]:
dataloader = DataLoader(data, batch_size=64, shuffle=False, num_workers=2, drop_last=True)
embeddings = []
labels = []

for imgs, labs in tqdm(dataloader):
    inputs = clip_processor(images=imgs, return_tensors="pt", padding=True, do_rescale=False).to(device)
    with torch.no_grad():
        outputs = clip_model(**inputs).pooler_output
        # curr = outputs.cpu()
    embeddings.append(outputs)
    labels.append(labs)

embeddings = torch.cat(embeddings)
labels = torch.cat(labels)

100%|██████████| 135/135 [01:22<00:00,  1.64it/s]


In [6]:
# Custom Dataset to Hold Embeddings
class EmbeddingData(Dataset):
    def __init__(self, embeddings, labels):
        self.embeddings = embeddings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, i):
        return self.embeddings[i], self.labels[i]

In [10]:
data = EmbeddingData(embeddings, labels)

In [19]:
torch.save(data, 'clip-data.pt')

## Splitting Data

In [20]:
torch.serialization.add_safe_globals([EmbeddingData])
# data = torch.load('clip-data.pt')
data = torch.load('clip-data.pt', map_location=device)

In [21]:
TRAIN_RATIO, VAL_RATIO = 0.7, 0.15

data_n = len(data)
train_n = int(TRAIN_RATIO * data_n)
val_n = int(VAL_RATIO * data_n)
test_n = data_n - train_n - val_n

In [22]:
train_data, val_data, test_data = random_split(
    data, [train_n, val_n, test_n], generator=torch.Generator().manual_seed(RAND_SEED)
)

In [23]:
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
val_loader = DataLoader(val_data, batch_size=64)
test_loader = DataLoader(test_data, batch_size=64)

# Modeling

## Creating Model

In [24]:
# Model Definition
class CLIPClassifier(L.LightningModule):
  def __init__(self, input_dim=768, hidden_dim=512, output_dim=101, lr=1e-3):
    super().__init__()
    self.lr = lr

    # Model Architecture
    self.model = torch.nn.Sequential(
        torch.nn.Linear(input_dim, hidden_dim),
        torch.nn.ReLU(),
        torch.nn.Dropout(0.2),
        torch.nn.Linear(hidden_dim, hidden_dim // 2),
        torch.nn.ReLU(),
        torch.nn.Dropout(0.2),
        torch.nn.Linear(hidden_dim // 2, output_dim)
    )

    self.loss_fn = torch.nn.CrossEntropyLoss()

  def forward(self, x):
    return self.model(x)

  def _step(self, batch, stage):
      x, y = batch
      logits = self.forward(x)
      loss = self.loss_fn(logits, y)

      preds = torch.argmax(logits, dim=1)
      acc = (preds == y).float().mean()

      self.log(f'{stage}_loss', loss, prog_bar=True)
      self.log(f'{stage}_acc', acc, prog_bar=True)

      return loss

  def training_step(self, batch, batch_idx):
      return self._step(batch, "train")

  def validation_step(self, batch, batch_idx):
      return self._step(batch, "val")

  def test_step(self, batch, batch_idx):
      return self._step(batch, "test")

  def configure_optimizers(self):
      optimizer = torch.optim.Adam(self.parameters(), lr=self.lr)
      scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
          optimizer, mode="min", factor=0.5, patience=2
      )
      return {
          "optimizer": optimizer,
          "lr_scheduler": {
              "scheduler": scheduler,
              "monitor": "val_loss",
              "interval": "epoch",
              "frequency": 1,
          },
      }

In [25]:
# Callbacks
checkpoint_callback = ModelCheckpoint(
    monitor="val_loss",
    dirpath="./checkpoints/",
    filename="clip-classifier-{epoch:02d}-{val_loss:.2f}",
    save_top_k=3,
    mode="min",
)

early_stop_callback = EarlyStopping(monitor="val_loss", patience=5, mode="min")

In [26]:
classifier = CLIPClassifier()

## Train Model

In [27]:
from lightning.pytorch.loggers import WandbLogger
import wandb

wandb.login(key=os.environ.get("WANDB_API_KEY"))

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [28]:
wandb_logger = WandbLogger(project="clip-classifier", name="clip-classifier", log_model=True)
trainer = L.Trainer(
    max_epochs=10,
    accelerator='auto',
    logger=wandb_logger,
    callbacks=[checkpoint_callback, early_stop_callback],
)

INFO: GPU available: False, used: False
INFO:lightning.pytorch.utilities.rank_zero:GPU available: False, used: False
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [29]:
trainer.fit(classifier, train_loader, val_loader)

/usr/local/lib/python3.11/dist-packages/lightning/pytorch/loggers/wandb.py:397: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
/usr/local/lib/python3.11/dist-packages/lightning/pytorch/callbacks/model_checkpoint.py:654: Checkpoint directory /content/drive/MyDrive/CLIPProj/checkpoints exists and is not empty.
INFO: 
  | Name    | Type             | Params | Mode 
-----------------------------------------------------
0 | model   | Sequential       | 551 K  | train
1 | loss_fn | CrossEntropyLoss | 0      | train
-----------------------------------------------------
551 K     Trainable params
0         Non-trainable params
551 K     Total params
2.204     Total estimated model params size (MB)
9         Modules in train mode
0         Modules in eval mode
INFO:lightning.pytorch.callbacks.model_summary:
  | Name    | Type             | Params | Mode 


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO: `Trainer.fit` stopped: `max_epochs=10` reached.
INFO:lightning.pytorch.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.


In [30]:
wandb.finish()

epoch,▁▁▂▂▃▄▄▄▅▅▆▆▆▆▆▇▇██▁▂▂▃▃▃▃▃▄▄▄▆▆▆▆▆▇▇▇██
train_acc,▁▅▄▆▇▇▇▅█▇▇▇▇▇█████▁▅▅▇▇▇▇▇▇█▇▇▇█▇▇███
train_loss,█▃▄▂▂▁▂▂▁▁▁▁▁▁▁▁▁▁▁▇▃▃▂▁▂▂▁▂▁▁▁▂▁▁▁▁▁▁
trainer/global_step,▁▁▁▂▂▃▄▄▅▅▆▆▆▆▇▇███▁▂▂▂▃▃▃▄▄▄▄▅▅▅▆▆▇▇▇██
val_acc,▂▅▆▆▆▇▆▇▇█▁▅▇▆▆▇▇▇▇█
val_loss,▇▄▂▂▂▁▂▂▂▁█▄▂▂▂▁▂▁▂▁
epoch,9
train_acc,1
train_loss,0.01328
trainer/global_step,949
val_acc,0.95833
